In [1]:
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
import re


In [ ]:
path = "chromdriver path\chromedriver.exe"
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.linkedin.com/login")
time.sleep(2)

# Enter your credentials
e = driver.find_element(by=By.ID, value="username")
e.send_keys("your email")
p = driver.find_element(by=By.ID, value="password")
p.send_keys("Your password")
button = driver.find_element(By.CSS_SELECTOR, ".btn__primary--large.from__button--floating")
button.click()
time.sleep(3)

url = "https://www.linkedin.com/posts/klarna_klarnas-climate-resilience-program-activity-7346877091532959746-748v/?utm_source=share&utm_medium=member_desktop&rcm=ACoAAADnFyQBBmDIWAAFjnAInMjU44QmY2tSbC8"
driver.get(url)
response = driver.page_source
post_div = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'div.update-components-text.relative.update-components-update-v2__commentary')
    )
)

# Get the post text 
post_text = post_div.text


# Click the like number
img_button = driver.find_element(By.CSS_SELECTOR, ".t-black--light.display-flex.align-items-center.social-details-social-counts__count-value.social-details-social-counts__count-value-hover.text-body-small.hoverable-link-text")
img_button.click()


WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.artdeco-list.artdeco-list--offset-1"))
)


while True:
    # Show more results button click
    try:
        load_more = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 
                "button.artdeco-button.artdeco-button--muted.artdeco-button--1.artdeco-button--full.artdeco-button--secondary.ember-view.scaffold-finite-scroll__load-button"))
        )
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(3)
    except TimeoutException:
        print("No more Load More button found, all results loaded.")
        break
    except ElementClickInterceptedException:
        print("Click intercepted, retrying after a short wait.")
        time.sleep(2)
    except NoSuchElementException:
        print("Load More button no longer present.")
        break


liker_items = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        "ul.artdeco-list.artdeco-list--offset-1 li.social-details-reactors-tab-body-list-item"))
)

print(f"Found {len(liker_items)} likers.")

output_data = {}

for id, liker in enumerate(liker_items):
    try:
        name = liker.find_element(By.CSS_SELECTOR, "span.text-view-model").text
        headline = liker.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__caption.ember-view").text
        output_data[id] = {
            "name": name,
            "headline": headline
        }
    except Exception:
        company_name = liker.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__title.ember-view").text
        output_data[id] = {
            "company_name": company_name
        }
len(output_data)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:

import csv

print(output_data)

# Save output_data to CSV with columns: name, title, company
with open('output_linkedin.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['name', 'title', 'company'])
    writer.writeheader()
    for entry in output_data.values():
        row = {
            'name': entry.get('name', ''),
            'title': entry.get('headline', ''),
            'company': entry.get('company_name', '')
        }
        writer.writerow(row)

{0: {'name': 'Colin Creagh', 'headline': 'Head of Klarna, Ireland | GTM expert | ex eBay | MBA'}, 1: {'name': 'Tori Galloso May', 'headline': 'Helping Cloud ISVs & GSIs to Craft Joint Value Propositions | GTM with AWS, Azure & GCP Hyperscalers ☁️ | Strategic Alliances'}, 2: {'name': 'Peter Githinji', 'headline': 'Driving Climate Action|Low Carbon Development Initiatives| Climate Finance & Carbon Markets Leader | Driving Sustainable Innovation & Economic Opportunity | Project Development & Impact Strategy'}, 3: {'name': 'Mike Alves, MSAFP, CFP®, CEPA, CRPC®', 'headline': 'Helping high net worth families align wealth with their values and live a purposeful life.'}, 4: {'name': 'Sara Simmonds', 'headline': 'CEO Impact Innovator | Helping Purpose Driven Entrepreneurs Scale to $1 Million, $50 Million and Beyond | Follow Me For Systems On Sustainable Growth 🌎'}, 5: {'company_name': 'pi2Life'}, 6: {'company_name': 'Funder'}, 7: {'name': 'Dhanesh Kothari', 'headline': 'Compassionate Capitalism

In [ ]:
import pandas as pd
# Read CSV from string (replace with read_csv from file)
from io import StringIO
df = pd.read_csv('read_the_csv_file_generated')

# Drop the 'company' column and remove rows with blank name or title
df = df[['name', 'title']]
df = df[(df['name'].notnull()) & (df['title'].notnull()) & (df['name'] != '') & (df['title'] != '')]

# Define keywords and scoring categories
functions = ['Sustainability', 'Climate', 'Innovation', 'Finance', 'GTM', 'Engineering', 'Marketing', 'Product', 'Tech', 'Strategic Alliances', 'Data']
seniorities = ['Head', 'Director', 'Manager', 'Lead', 'Chief', 'CEO', 'SVP', 'President', 'Founder', 'Architect']
nears = ['Senior', 'Account', 'Delivery', 'Business Development', 'Consultant', 'Controller']
company_types = ['Klarna', 'FinTech', 'RetailTech', 'Tech', 'Blue Earth', 'IBM', 'Qliro', 'PayPal', 'Hexagon', 'Norrsken', 'Doconomy', 'Milkywire', 'Oney', 'Planboo', 'Johnson Real Estate', 'AJR.Agency']
geos = ['Ireland', 'UK', 'ANZ', 'Nordics', 'Europe', 'Spain', 'Portugal', 'Sweden']
keywords = ['Climate', 'Sustainable', 'Carbon', 'ESG', 'Impact']

# Scoring function
def relevance_score(row):
    score = 0
    reason = []
    title_lower = row['title'].lower()
    
    # Function match (exact and partial)
    if any(f.lower() in title_lower for f in functions):
        score += 40
        reason.append('Function')
    elif any(any(w.lower() in title_lower for w in f.lower().split()) for f in functions):
        score += 20
        reason.append('Partial Function')
    
    # Seniority match
    if any(s in row['title'] for s in seniorities):
        score += 25
        reason.append('Seniority')
    elif any(n in row['title'] for n in nears):
        score += 10
        reason.append('Near Seniority')
    
    # Company type match (only title here since company dropped)
    if any(ct.lower() in title_lower for ct in company_types):
        score += 20
        reason.append('Type')
    elif any(any(w.lower() in title_lower for w in ct.lower().split()) for ct in company_types):
        score += 10
        reason.append('Adj Type')
    
    # Geo match
    if any(g in row['title'] for g in geos):
        score += 10
        reason.append('Geo')
    
    # Bonus keywords
    kw_matches = sum(1 for k in keywords if k.lower() in title_lower)
    if kw_matches > 0:
        score += min(10, kw_matches * 5)
        reason.append('Keyword')
    
    # Cap score at 100
    score = min(score, 100)
    return pd.Series([score, '+'.join(reason), False], index=['score', 'score_reason', 'excluded'])

# Apply scoring
df[['score', 'score_reason', 'excluded']] = df.apply(relevance_score, axis=1)



# Show results



In [ ]:
# Save DataFrame to CSV
df.to_csv('scored_output_linkedin.csv', index=False)

In [ ]:
print(post_text)

🌍 We’re proud to announce the launch of our AI for Climate Resilience Program.

AI already powers everything we do at Klarna — and now we’re turning that same expertise toward the front lines of climate change. We take pride in our legacy as a climate leader, and we’re committed to driving positive change for the future. The AI for Climate Resilience Program will support pioneering projects that harness artificial intelligence to help climate-vulnerable communities adapt and thrive.

This is technology in service of both people and the planet.

This program will support local, practical, and community-owned solutions. From strengthening food security and improving health systems to building coastal resilience in the face of climate change.

What’s on offer:
💸 Grants of up to $300,000
🧑‍🎓 Mentorship, training, and a supportive community of practice

We encourage applications from organizations working to reduce vulnerability of local communities to climate-related risks in low- and midd